In [123]:
from keras.datasets import mnist
import numpy as np
np.random.seed(10)

from time import time
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec

from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from sklearn import metrics
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose

from encoder_function import autoencoder, autoencoderConv2D_1, autoencoderConv2D_2, ClusteringLayer


## Load Data

In [124]:
import cv2
from keras.preprocessing.image import img_to_array
import os
root_path = './label_6/'
images = []
labels = []
for folder in os.listdir(root_path):
    if 'DS_Store' in folder:
        continue
    for f_name in os.listdir(os.path.join(root_path, folder)):
        if 'DS_Store' in f_name:
            continue
        image = cv2.imread(os.path.join(root_path, folder, f_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (28,28))
        image = image.astype("float") / 255.0
        image = img_to_array(image)
        images.append(image)
        labels.append(folder)

images = np.array(images)
x = images.reshape((images.shape[0], -1))
y = labels
print(x.shape)
n_clusters = len(np.unique(y))
print(n_clusters)


(5373, 784)
6


## Baseline K Means Clustering Accuracy

In [118]:
kmeans = KMeans(n_clusters=n_clusters, n_init=6, n_jobs=-1)
y_pred_kmeans = kmeans.fit_predict(x)

In [119]:
y_pred_kmeans

array([4, 3, 3, ..., 1, 4, 5], dtype=int32)

In [120]:
print(metrics.mutual_info_score(y,y_pred_kmeans))
metrics.adjusted_rand_score(y,y_pred_kmeans)

0.43605725889679814


0.12347674555865912

## Pre-train Auto-encoder

In [125]:
dims = [x.shape[-1], 500, 500, 2000, 6]
init = VarianceScaling(scale=1. / 3., mode='fan_in',distribution='uniform')
pretrain_optimizer = SGD(lr=0.01, momentum=0.9)
pretrain_epochs = 600
batch_size = 200
save_dir = './results/screen_clustering/'


In [126]:
autoencoder1, encoder1 = autoencoder(dims, init=init)
autoencoder1.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder1.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)
autoencoder1.save_weights(save_dir + '/ae_weights.h5')
autoencoder1.load_weights(save_dir + '/ae_weights.h5')

Epoch 1/600
5373/5373 [==============================] - 1s 244us/step - loss: 0.5756
Epoch 2/600
5373/5373 [==============================] - 0s 22us/step - loss: 0.5699
Epoch 3/600
5373/5373 [==============================] - 0s 21us/step - loss: 0.5634
Epoch 4/600
5373/5373 [==============================] - 0s 21us/step - loss: 0.5563
Epoch 5/600
5373/5373 [==============================] - 0s 22us/step - loss: 0.5471
Epoch 6/600
5373/5373 [==============================] - 0s 22us/step - loss: 0.5248
Epoch 7/600
5373/5373 [==============================] - 0s 21us/step - loss: 0.3492
Epoch 8/600
5373/5373 [==============================] - 0s 21us/step - loss: 0.1363
Epoch 9/600
5373/5373 [==============================] - 0s 21us/step - loss: 0.1269
Epoch 10/600
5373/5373 [==============================] - 0s 21us/step - loss: 0.1261
Epoch 11/600
5373/5373 [==============================] - 0s 22us/step - loss: 0.1260
Epoch 12/600
5373/5373 [==============================] - 0s 2

## Clustering Layer

In [127]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder1.output)
model = Model(inputs=encoder1.input, outputs=clustering_layer)
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')


### step1. initialize cluster centers with kmeans

In [128]:
kmeans = KMeans(n_clusters=n_clusters, n_init=6)
y_pred = kmeans.fit_predict(encoder1.predict(x))

y_pred_last = np.copy(y_pred)
print('Clustering NMI after encoding:%f'% metrics.mutual_info_score(y, y_pred_last))
print('Clustering ARI after encoding:%f'% metrics.adjusted_rand_score(y, y_pred_last))

Clustering NMI after encoding:0.295065
Clustering ARI after encoding:0.074433


In [129]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

### step2.Deep Clustering

In [131]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

loss = 0
index = 0
maxiter = 16000
update_interval = 140
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training

# start training

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: nmi = %.5f, ari = %.5f' % (ite, nmi, ari), ' ; loss=', loss)

        # check stop criterion - model convergence
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/DEC_model_final.h5')

('Iter 0: nmi = 0.34749, ari = 0.07077', ' ; loss=', 0)
('Iter 140: nmi = 0.34225, ari = 0.07160', ' ; loss=', 0.04744)
('Iter 280: nmi = 0.35346, ari = 0.09108', ' ; loss=', 0.13827)
('Iter 420: nmi = 0.34572, ari = 0.07966', ' ; loss=', 0.18434)
('Iter 560: nmi = 0.34961, ari = 0.07866', ' ; loss=', 0.25351)
('Iter 700: nmi = 0.34420, ari = 0.07750', ' ; loss=', 0.10131)
('Iter 840: nmi = 0.32646, ari = 0.07448', ' ; loss=', 0.07995)
('Iter 980: nmi = 0.30735, ari = 0.06782', ' ; loss=', 0.48376)
('Iter 1120: nmi = 0.30934, ari = 0.08077', ' ; loss=', 0.13933)
('Iter 1260: nmi = 0.32932, ari = 0.09968', ' ; loss=', 0.38518)
('Iter 1400: nmi = 0.30948, ari = 0.09539', ' ; loss=', 0.30549)
('Iter 1540: nmi = 0.32764, ari = 0.10475', ' ; loss=', 0.05691)
('Iter 1680: nmi = 0.34407, ari = 0.11653', ' ; loss=', 0.08286)
('Iter 1820: nmi = 0.35134, ari = 0.12103', ' ; loss=', 0.22678)
('Iter 1960: nmi = 0.34070, ari = 0.11497', ' ; loss=', 0.11848)
('Iter 2100: nmi = 0.35669, ari = 0.10259

In [132]:
model.load_weights(save_dir + '/DEC_model_final.h5')
# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)


('nmi = 0.42028, ari = 0.16912', ' ; loss=', 0.06358)


## Experiment 2. Convolutional Auto-encoder

In [1]:
# x = images.reshape(images.shape + (1,))
# images.shape
x = images

NameError: name 'images' is not defined

In [135]:
def autoencoderConv2D_1(input_shape=(28, 28, 1), filters=[32, 64, 128, 10]):
    input_img = Input(shape=input_shape)
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)

    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)

    x = Conv2D(filters[2], 3, strides=2, padding=pad3, activation='relu', name='conv3')(x)

    x = Flatten()(x)
    encoded = Dense(units=filters[3], name='embedding')(x)
    x = Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu')(encoded)

    x = Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2]))(x)
    x = Conv2DTranspose(filters[1], 3, strides=2, padding=pad3, activation='relu', name='deconv3')(x)

    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1')(x)
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

autoencoder2, encoder2 = autoencoderConv2D_1(input_shape=(28, 28, 1), filters=[32, 64, 128, 10])
pretrain_epochs = 600
batch_size = 200
autoencoder2.compile(optimizer='adadelta', loss='mse')
autoencoder2.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs)
autoencoder2.save_weights(save_dir+'/conv_ae_weights.h5')

Epoch 1/600
5373/5373 [==============================] - 2s 371us/step - loss: 0.3638
Epoch 2/600
5373/5373 [==============================] - 0s 40us/step - loss: 0.1911
Epoch 3/600
5373/5373 [==============================] - 0s 39us/step - loss: 0.1778
Epoch 4/600
5373/5373 [==============================] - 0s 40us/step - loss: 0.1695
Epoch 5/600
5373/5373 [==============================] - 0s 37us/step - loss: 0.1644
Epoch 6/600
5373/5373 [==============================] - 0s 39us/step - loss: 0.1547
Epoch 7/600
5373/5373 [==============================] - 0s 39us/step - loss: 0.1522
Epoch 8/600
5373/5373 [==============================] - 0s 40us/step - loss: 0.1510
Epoch 9/600
5373/5373 [==============================] - 0s 39us/step - loss: 0.1430
Epoch 10/600
5373/5373 [==============================] - 0s 39us/step - loss: 0.1353
Epoch 11/600
5373/5373 [==============================] - 0s 39us/step - loss: 0.1330
Epoch 12/600
5373/5373 [==============================] - 0s 3

In [136]:
autoencoder2.load_weights(save_dir+'/conv_ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder2.output)
model = Model(inputs=encoder2.input, outputs=clustering_layer)
model.compile(optimizer='adam', loss='kld')

kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder2.predict(x))
y_pred_last = np.copy(y_pred)
print(metrics.mutual_info_score(y, y_pred_last))
print ('Convolutional Auto-encoder 1 has ARI: %f' % metrics.adjusted_rand_score(y,y_pred_last))

0.6639839888491164
Convolutional Auto-encoder 1 has ARI: 0.285273


In [137]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
loss = 0
index = 0
maxiter = 16000
update_interval = 300
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training

# start training
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d:nmi = %.5f, ari = %.5f' % (ite,nmi, ari), ' ; loss=', loss)

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/conv_DEC_model_final.h5')

#Load the clustering model trained weights
model.load_weights(save_dir + '/conv_DEC_model_final.h5')
# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)



('Iter 0:nmi = 0.66398, ari = 0.28527', ' ; loss=', 0)
('Iter 300:nmi = 0.66582, ari = 0.28591', ' ; loss=', 0.01879)
('Iter 600:nmi = 0.66122, ari = 0.28732', ' ; loss=', 0.0374)
('Iter 900:nmi = 0.70110, ari = 0.30428', ' ; loss=', 0.10123)
('Iter 1200:nmi = 0.68438, ari = 0.30360', ' ; loss=', 0.0846)
('Iter 1500:nmi = 0.69733, ari = 0.31076', ' ; loss=', 0.04388)
('Iter 1800:nmi = 0.70787, ari = 0.31437', ' ; loss=', 0.32311)
('Iter 2100:nmi = 0.70670, ari = 0.31565', ' ; loss=', 0.06742)
('Iter 2400:nmi = 0.71533, ari = 0.31918', ' ; loss=', 0.06896)
('Iter 2700:nmi = 0.71504, ari = 0.31905', ' ; loss=', 0.06837)
('Iter 3000:nmi = 0.71781, ari = 0.32043', ' ; loss=', 0.04501)
('Iter 3300:nmi = 0.72123, ari = 0.32291', ' ; loss=', 0.04325)
('Iter 3600:nmi = 0.72278, ari = 0.32230', ' ; loss=', 0.03168)
('Iter 3900:nmi = 0.72290, ari = 0.32203', ' ; loss=', 0.0352)
('Iter 4200:nmi = 0.72160, ari = 0.32174', ' ; loss=', 0.02838)
('Iter 4500:nmi = 0.72139, ari = 0.32217', ' ; loss=', 

In [138]:
delta_label

0.000744463056020845

## Experiment 3. Model to train Autoencoder and clustering layer together (Conv)

In [139]:
autoencoder, encoder = autoencoderConv2D_1()
autoencoder.load_weights(save_dir+'/conv_ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])

In [140]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
y_pred_last = np.copy(y_pred)

In [142]:

loss = 0
index = 0
maxiter = 16000
update_interval = 300
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer='adam')
# start training
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _  = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d:nmi = %.5f, ari = %.5f' % (ite,nmi, ari), ' ; loss=', loss)

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=[p[idx], x[idx]])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/conv_b_DEC_model_final.h5')





('Iter 0:nmi = 0.66376, ari = 0.28183', ' ; loss=', 0)
('Iter 300:nmi = 0.65132, ari = 0.25941', ' ; loss=', array([0.03009, 0.02939, 0.02715], dtype=float32))
('Iter 600:nmi = 0.66912, ari = 0.26629', ' ; loss=', array([0.04996, 0.06818, 0.04314], dtype=float32))
('Iter 900:nmi = 0.67831, ari = 0.27690', ' ; loss=', array([0.07052, 0.10286, 0.06024], dtype=float32))
('Iter 1200:nmi = 0.68289, ari = 0.28785', ' ; loss=', array([0.06975, 0.10839, 0.05891], dtype=float32))
('Iter 1500:nmi = 0.68355, ari = 0.29493', ' ; loss=', array([0.06498, 0.12008, 0.05298], dtype=float32))
('Iter 1800:nmi = 0.69049, ari = 0.29709', ' ; loss=', array([0.0536 , 0.18909, 0.03469], dtype=float32))
('Iter 2100:nmi = 0.69186, ari = 0.29980', ' ; loss=', array([0.07475, 0.11607, 0.06314], dtype=float32))
('Iter 2400:nmi = 0.69788, ari = 0.30148', ' ; loss=', array([0.08713, 0.12205, 0.07493], dtype=float32))
('Iter 2700:nmi = 0.70306, ari = 0.30257', ' ; loss=', array([0.03493, 0.11506, 0.02342], dtype=floa

In [143]:
model.load_weights(save_dir + '/conv_b_DEC_model_final.h5')

q, _ = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)



('nmi = 0.74213, ari = 0.32059', ' ; loss=', array([0.04461, 0.06167, 0.03844], dtype=float32))


## Experiment 4. Model to train Autoencoder and clustering layer together (Fully connected)

In [144]:
x = images.reshape((images.shape[0], -1))
x.shape

(5373, 784)

In [145]:
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(lr=0.01, momentum=0.9)
pretrain_epochs = 600
batch_size = 200
save_dir = './results'


from encoder_function import autoencoder
autoencoder, encoder = autoencoder(dims, init=init)
autoencoder.load_weights(save_dir+'/ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input,
            outputs=[clustering_layer, autoencoder.output])

In [146]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
y_pred_last = np.copy(y_pred)

In [147]:
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer=pretrain_optimizer)

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _  = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d:nmi = %.5f, ari = %.5f' % (ite,nmi, ari), ' ; loss=', loss)

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=[p[idx], x[idx]])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/b_DEC_model_final.h5')



('Iter 0:nmi = 0.28587, ari = 0.09329', ' ; loss=', array([0.04461, 0.06167, 0.03844], dtype=float32))
('Iter 300:nmi = 0.30203, ari = 0.09620', ' ; loss=', array([0.12691, 0.08436, 0.11847], dtype=float32))
('Iter 600:nmi = 0.30663, ari = 0.09980', ' ; loss=', array([0.10021, 0.13862, 0.08635], dtype=float32))
('Iter 900:nmi = 0.31579, ari = 0.10498', ' ; loss=', array([0.126  , 0.14096, 0.1119 ], dtype=float32))
('Iter 1200:nmi = 0.32852, ari = 0.10898', ' ; loss=', array([0.14516, 0.15443, 0.12972], dtype=float32))
('Iter 1500:nmi = 0.33639, ari = 0.11142', ' ; loss=', array([0.05179, 0.14722, 0.03706], dtype=float32))
('Iter 1800:nmi = 0.34449, ari = 0.11414', ' ; loss=', array([0.05478, 0.13686, 0.04109], dtype=float32))
('Iter 2100:nmi = 0.35333, ari = 0.11830', ' ; loss=', array([0.09267, 0.15346, 0.07732], dtype=float32))
('Iter 2400:nmi = 0.35540, ari = 0.12115', ' ; loss=', array([0.10023, 0.14199, 0.08603], dtype=float32))
('Iter 2700:nmi = 0.35148, ari = 0.11965', ' ; loss=

In [148]:
model.load_weights(save_dir + '/b_DEC_model_final.h5')
q, _ = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)



('nmi = 0.39003, ari = 0.13350', ' ; loss=', array([0.03656, 0.13965, 0.02259], dtype=float32))
